<a href="https://colab.research.google.com/github/alunfes/1m-btc-data/blob/master/DQN3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%load_ext Cython
#%load_ext tensorboard
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)
!pip install optuna
!pip install six
!pip install stable-baselines --upgrade
!pip install tensorboardcolab


from sklearn import datasets
import dateutil
import copy
import sqlite3
import math
import warnings
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import pickle
import csv
import matplotlib.pyplot as plt
from scipy.spatial.distance import correlation
import numpy as np
import pandas as pd
import multiprocessing as multi
from datetime import datetime, timedelta, timezone
import sklearn.metrics
from sklearn.model_selection import train_test_split
from functools import partial
import optuna
import time
import random
import json
import gym
from gym import spaces
from stable_baselines.bench import Monitor
from stable_baselines.common.policies import MlpPolicy, MlpLnLstmPolicy, CnnLnLstmPolicy
from stable_baselines.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines import PPO2, TRPO, ACKTR
from tensorboardcolab import *

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython
1.15.2


In [ ]:

class MarketData:
    @classmethod
    def initialize(cls, ma_term):
        print('Reading Market Data')
        cls.ma_term = ma_term
        cls.ohlcv = cls.readData()
        cls.open = list(cls.ohlcv['open'])
        cls.high = list(cls.ohlcv['high'])
        cls.low = list(cls.ohlcv['low'])
        cls.close = list(cls.ohlcv['close'])
        cls.vol = list(cls.ohlcv['volume'])
        cls.ts = list(cls.ohlcv['ts'])
        cls.ohlcv['ma'] = cls.calcMA(ma_term)
        cls.ohlcv['change'] = cls.calcChange()
        print('completed initialize market data')
        print(cls.ohlcv[0:10])
    
    @classmethod
    def readData(cls):
        return pd.read_csv('/content/drive/My Drive/BTC-PERP-1m.csv',index_col=0)
    
    @classmethod
    def calcMA(cls, ma_term):
        return cls.ohlcv['close'].rolling(window = ma_term).mean()
    
    @classmethod

    def calcChange(cls):
        return (cls.ohlcv['close'] - cls.ohlcv['open']) / cls.ohlcv['open']
                
MarketData.initialize(10)

Reading Market Data
completed initialize market data
              ts      open      high       low     close  volume        ma  \
0  1564814820000  10790.00  10793.25  10787.75  10793.25     0.0       NaN   
1  1564814880000  10793.25  10803.00  10790.00  10795.75     0.0       NaN   
2  1564814940000  10795.75  10806.25  10795.75  10795.75     0.0       NaN   
3  1564815000000  10795.75  10798.00  10794.50  10797.75     0.0       NaN   
4  1564815060000  10797.75  10809.75  10794.50  10809.75     0.0       NaN   
5  1564815120000  10809.75  10811.25  10794.50  10798.25     0.0       NaN   
6  1564815180000  10798.25  10803.75  10798.25  10802.00     0.0       NaN   
7  1564815240000  10802.00  10802.25  10798.25  10798.25     0.0       NaN   
8  1564815300000  10798.25  10800.75  10798.25  10798.25     0.0       NaN   
9  1564815360000  10798.25  10804.75  10794.50  10803.50     0.0  10799.25   

     change  
0  0.000301  
1  0.000232  
2  0.000000  
3  0.000185  
4  0.001111  
5 -0

In [ ]:
'''
unput data:
minmaxscaled(ma[0:term]), current change ratio, current position
'''


class MyEnv(gym.Env):
    metadata = {'render.modes': ['console']}
    def __init__(self, ohlcv_df, ma_length, start_ind):
        super(MyEnv, self).__init__()
        self.scaler = MinMaxScaler()
        self.ohlcv_df = ohlcv_df.reset_index(drop=True)
        self.ma_length = ma_length
        self.taker_fee = 0.005
        self.slip_page = 0.005
        self.limit_stop_loss = -0.2
        self.action_space = spaces.Discrete(2) #change or keep
        self.observation_space = spaces.Box(low=-1, high=1, shape=(self.ma_length + 1 + 1,), dtype=np.float64)
        self.reset()


    def reset(self):
        self.i = 1000
        self.pre_reward = 0
        self.num_trade = 0
        self.current_position = 1 #1:long, -1:short
        self.total_pl = 0
        self.current_pl = 0
        self.pre_pl = [0]
        self.win_rate = 0
        self.num_win = 0
        self.entry_price = self.ohlcv_df['close'][self.i]
        return self._observe()


    def render(self, mode='console', close=False):
        print('i=', self.i, ', pl=', self.total_pl, ', num trade=', self.num_trade, ', win rate=', self.win_rate, ', current_pl=', self.current_pl)


    def _observe(self):
        ma = self.scaler.fit_transform(np.array(self.ohlcv_df['ma'].iloc[self.i - self.ma_length+1: self.i+1]).reshape(-1,1)).T
        input = np.append(ma[0], [self.ohlcv_df['change'].iloc[self.i], self.current_position])
        return input
    

    def _reward(self):
        return self.pre_pl[-2]


    def step(self, action):
        #calc current pl
        if self.current_position == 1:
            self.current_pl = (self.ohlcv_df['close'][self.i] - self.entry_price) / self.entry_price
        else:
            self.current_pl = (self.entry_price - self.ohlcv_df['close'][self.i]) / self.entry_price
        self.pre_pl.append(self.current_pl)
        #change position
        if self.current_position != action:
            pl = self.current_pl - self.taker_fee - self.slip_page
            if pl > 0:
                self.num_win += 1
            self.total_pl = self.total_pl + pl
            self.current_position = action
            self.entry_price = self.ohlcv_df['close'][self.i]
            self.num_trade +=1
            self.win_rate = round(self.num_win / self.num_trade, 4)
        self.i += 1
        return self._observe(), self._reward(), self._is_done(), {'pl':self.total_pl, 'num':self.num_trade, 'win rate':self.win_rate}
        
        
    def _is_done(self):
        if (self.total_pl <= self.limit_stop_loss) or self.i >= len(self.ohlcv_df)-1:
            return True
        else:
            return False



np.seterr(all="ignore")
warnings.simplefilter('ignore')
'''
env = MyEnv(MarketData.ohlcv.iloc[0:100000], 10, 1000)
state = env.reset()
total_reward = 0
while True:
    action = env.action_space.sample()
    state, reward, done, info = env.step(action)
    total_reward += reward
    env.render()
    print('reward:', reward, 'total_reward', total_reward)
    print('-----------')
    if done:
        print('done')
        break
'''
log_dir = '/content/drive/My Drive/Model/rl_log'
start_ind = 1000
num_train = 10000
env = MyEnv(MarketData.ohlcv.iloc[0:100000], 10, start_ind)
env = DummyVecEnv( [lambda: env] )
model = PPO2('MlpLnLstmPolicy', env,  verbose=1, nminibatches=1, tensorboard_log=log_dir)
model.learn(total_timesteps=num_train)
model.save('/content/drive/My Drive/Model/rf_ppo2')
env.close()


print('completed training')

'''
test_env = DummyVecEnv([lambda: MyEnv(MarketData.ohlcv.iloc[100000:110000], 10, start_ind)])
test_env.reset()
obs = test_env._observe()
print(state)
done = False
while done==False:
    action, _states = model.predict(obs)
    obs, rewards, done, performance = test_env.step(action)
    test_env.render()
'''

Streaming output truncated to the last 5000 lines.
| policy_entropy     | 0.0014088944  |
| policy_loss        | -2.484885e-05 |
| serial_timesteps   | 950784        |
| time_elapsed       | 1.19e+03      |
| total_timesteps    | 950784        |
| value_loss         | 0.15208563    |
--------------------------------------
--------------------------------------
| approxkl           | 0.0           |
| clipfrac           | 0.0           |
| explained_variance | 0.0238        |
| fps                | 834           |
| n_updates          | 7429          |
| policy_entropy     | 1.2030816e-07 |
| policy_loss        | -7.334165e-08 |
| serial_timesteps   | 950912        |
| time_elapsed       | 1.19e+03      |
| total_timesteps    | 950912        |
| value_loss         | 0.40644336    |
--------------------------------------
---------------------------------------
| approxkl           | 0.0            |
| clipfrac           | 0.0            |
| explained_variance | -0.161         |
| fps    

'\ntest_env = DummyVecEnv([lambda: MyEnv(MarketData.ohlcv.iloc[100000:110000], 10, start_ind)])\ntest_env.reset()\nobs = test_env._observe()\nprint(state)\ndone = False\nwhile done==False:\n    action, _states = model.predict(obs)\n    obs, rewards, done, performance = test_env.step(action)\n    test_env.render()\n'

In [ ]:
test_env = DummyVecEnv([lambda: MyEnv(MarketData.ohlcv.iloc[0:100000], 10, start_ind)])
obs = test_env.reset()
done = False
while done==False:
    action, _states = model.predict(obs)
    obs, rewards, done, performance = test_env.step(action)
    test_env.render()

i= 1001 , pl= -0.01 , num trade= 1 , win rate= 0.0 , current_pl= 0.0
i= 1002 , pl= -0.01 , num trade= 1 , win rate= 0.0 , current_pl= -0.00013852014313748125
i= 1003 , pl= -0.01 , num trade= 1 , win rate= 0.0 , current_pl= -0.00013852014313748125
i= 1004 , pl= -0.01 , num trade= 1 , win rate= 0.0 , current_pl= 0.0007849474777790604
i= 1005 , pl= -0.01 , num trade= 1 , win rate= 0.0 , current_pl= 0.0011312478356227634
i= 1006 , pl= -0.01 , num trade= 1 , win rate= 0.0 , current_pl= 0.0010158143830081958
i= 1007 , pl= -0.01 , num trade= 1 , win rate= 0.0 , current_pl= 0.0013852014313748124
i= 1008 , pl= -0.01 , num trade= 1 , win rate= 0.0 , current_pl= 0.0013852014313748124
i= 1009 , pl= -0.01 , num trade= 1 , win rate= 0.0 , current_pl= 0.0016160683366039479
i= 1010 , pl= -0.01 , num trade= 1 , win rate= 0.0 , current_pl= 0.0027011427911808844
i= 1011 , pl= -0.01 , num trade= 1 , win rate= 0.0 , current_pl= 0.002424102504905922
i= 1012 , pl= -0.01 , num trade= 1 , win rate= 0.0 , curre

In [ ]:
df = MarketData.ohlcv.iloc[100000:110000]
df = df.reset_index(drop=True)
df.iloc[1000:1010]

In [ ]:
MarketData.ma_term

10

In [ ]:
scaler = MinMaxScaler()
i = 1000
ma_length = 10
ma = scaler.fit_transform(np.array(MarketData.ohlcv['ma'].iloc[i - ma_length+1: i+1]).reshape(-1,1)).T
con = np.append(ma[0], [1,2])

In [ ]:
con.shape

(12,)

In [ ]:
class GoLeftEnv(gym.Env):
  """
  Gymのインターフェースに従うカスタム環境
  エージェントが常に左に行くことを学ぶ環境
  """
  # ColabのためGUIを実装できない
  metadata = {'render.modes': ['console']}

  # 定数を定義
  LEFT = 0
  RIGHT = 1

  def __init__(self, grid_size=10):
    super(GoLeftEnv, self).__init__()

    # 1Dグリッドのサイズ
    self.grid_size = grid_size

    # グリッドの右側でエージェントを初期化
    self.agent_pos = grid_size - 1

    # 行動空間と状態空間を定義
    # gym.spacesオブジェクトでなければならない
    # 離散行動を使用する場合の例には、左と右の2つがある
    n_actions = 2
    self.action_space = spaces.Discrete(n_actions)

    # 状態はエージェントの座標になる
    # Discrete空間とBox空間の両方で表現できる
    self.observation_space = spaces.Box(low=0, high=self.grid_size,
                                       shape=(1,), dtype=np.float32)

  def reset(self):
    """
    【重要】観測はnumpy配列でなければならない
    :return: (np.array)
    """
    # グリッドの右側でエージェントを初期化
    self.agent_pos = self.grid_size - 1

    # float32に変換してより一般的なものにします（連続行動を使用する場合）
    return np.array(self.agent_pos).astype(np.float32)

  def step(self, action):
    if action == self.LEFT:
      self.agent_pos -= 1
    elif action == self.RIGHT:
      self.agent_pos += 1
    else:
      raise ValueError("Received invalid action={} which is not part of the action space".format(action))

    # グリッドの境界を表現
    self.agent_pos = np.clip(self.agent_pos, 0, self.grid_size)

    # グリッドの左側にいるか
    done = self.agent_pos == 0

    # ゴールを除くすべての場所で0の報酬
    reward = 1 if self.agent_pos == 0 else 0

    # 必要に応じて情報を渡すことができるが、現在は未使用
    info = {}

    return np.array(self.agent_pos).astype(np.float32), reward, done, info

  def render(self, mode='console', close=False):
    if mode != 'console':
      raise NotImplementedError()

    # エージェントは「x」、残りは「.」として表現
    print("." * self.agent_pos, end="")
    print("x", end="")
    print("." * (self.grid_size - self.agent_pos))

In [ ]:
env = GoLeftEnv(grid_size=10)

obs = env.reset()
env.render()

print(env.observation_space)
print(env.action_space)
print(env.action_space.sample())

# 環境の生成
env = GoLeftEnv(grid_size=10)

# 環境のラップ
env = Monitor(env, filename=None, allow_early_resets=True)
env = DummyVecEnv([lambda: env])
model = ACKTR('MlpPolicy', env, verbose=1).learn(5000)

.........x.
Box(0.0, 10.0, (1,), float32)
Discrete(2)
1













---------------------------------
| explained_variance | 0.0267   |
| fps                | 24       |
| nupdates           | 1        |
| policy_entropy     | 0.693    |
| policy_loss        | 0.0634   |
| total_timesteps    | 20       |
| value_loss         | 0.0132   |
---------------------------------
---------------------------------
| ep_len_mean        | 30.7     |
| ep_reward_mean     | 1        |
| explained_variance | 0.192    |
| fps                | 464      |
| nupdates           | 100      |
| policy_entropy     | 0.188    |
| policy_loss        | 0.0853   |
| total_timesteps    | 2000     |
| value_loss         | 0.13     |
---------------------------------
---------------------------------
| ep_len_mean        | 10.5     |
| ep_reward_mean     | 1        |
| explained_variance | 0.873    |
| fps                | 486      |
| nupdates           | 200      |
| policy_entropy     | 0.236    |
| policy_loss

In [ ]:
# 環境の生成
env = GoLeftEnv(grid_size=10)

# 環境のラップ
env = Monitor(env, filename=None, allow_early_resets=True)
env = DummyVecEnv([lambda: env])